2025-11-25. Demo of restructured mass spectrum simulation project

This notebook tests my scripts for a small subset of the molecules in my real dataset. 
In the future, we can use this for a workflow demo. 

## 0. Create a toy dataset as a subset from real dataset

In [ ]:
import pandas as pd

# Load full dataset
df = pd.read_csv('../data/raw/Franklin/goamazon.csv')

# Pick 4 molecules by index:
toy_indices = [0, 1, 2, 3]  # first 4 rows for simplicity
toy_df = df.iloc[toy_indices].copy()

# Duplicate 2 of them, say first and third
duplicates = toy_df.iloc[[0, 2]].copy()
toy_df = pd.concat([toy_df, duplicates], ignore_index=True)

# Save toy dataset
toy_df.to_csv('../data/raw/toy_data/toy_dataset.csv', index=False)

## 1. Remove duplicates from dataset and write smiles in canonical form. Output: A list of SMILES to use as input for derivitization. 

In [ ]:
! . ~/.bashrc
!echo $SCRIPTS_NEIMS/processing

In [ ]:
%run /scratch/project_2006752/hsandstr/Project/atmospheric-ms-benchmark/src/processing/remove_duplicate_SMILES_entries.py -i ../data/raw/toy_data/toy_dataset.csv -o ../data/processed/toy_data/toy_dataset_unique.csv  --log_file ../data/processed/toy_data/toy_dataset_duplicates.csv


## 2.1 Derivatize molecules. Output: A list of derivatized SMILES for input to MS simulations

In [ ]:
%run /scratch/project_2006752/hsandstr/Project/atmospheric-ms-benchmark/src/processing/make_TMS_derivative_251125_v1.py --compare_ref \
    -i ../data/processed/toy_data/toy_dataset_unique.csv \
    -o ../data/processed/toy_data_tms/toy_dataset_TMS.csv


## 3. Make molecule folders and run sdf from smiles

In [2]:
%run /scratch/project_2006752/hsandstr/Project/atmospheric-ms-benchmark/src/processing/make_directories_and_sdfs.py  --input_csv ../data/processed/toy_data_tms/toy_dataset_TMS.csv --output_root ../data/simulation_results/toy_dataset_tms


[EXISTING] Folder exists: 0000
  → SMILES file exists, skipping
  → Generating SDF
  → metadata.json exists, skipping
[EXISTING] Folder exists: 0001
  → SMILES file exists, skipping
  → Generating SDF
  → metadata.json exists, skipping
[EXISTING] Folder exists: 0002
  → SMILES file exists, skipping
  → Generating SDF
  → metadata.json exists, skipping
[EXISTING] Folder exists: 0003
  → SMILES file exists, skipping
  → Generating SDF
  → metadata.json exists, skipping


## 4.1 Run the initial QCxMS ground state optimization and MD

In [4]:
import os
import subprocess

sim_dir = os.path.abspath(os.path.join(os.getcwd(), "../data/simulation_results/toy_dataset_tms/QCxMS_v3/"))
script_path = os.path.abspath(os.path.join(sim_dir, "../../../../src/workflow/submit_qcxms_gs_md.sh"))  # adjust path to SLURM script

result = subprocess.run(
    ["sbatch", "--array=0-3", script_path],
    cwd=sim_dir,  # run from toy_data_tms
    capture_output=True,
    text=True
)

print(result.stdout)
print(result.stderr)


Submitted batch job 30857826




## 4.2 Run QCxMS fragmentation script for each molecule

In [2]:
import os
import subprocess

# Root simulation directory
sim_dir = "../data/simulation_results/toy_dataset_tms"
script_path = os.path.abspath(os.path.join(sim_dir, "../../../src/workflow/submit_qcxms_frag_serial.sh"))
# Find all molecule folders (assumes numeric names)
molecule_folders = sorted([f for f in os.listdir(sim_dir) if f.isdigit()])

for mol in molecule_folders:
    mol_path = os.path.join(sim_dir, mol, "QCxMS", "MS-run")

    if not os.path.isdir(mol_path):
        print(f"Skipping {mol}, QCxMS/MS-run folder not found")
        continue


    # Run the SLURM submission script
    result = subprocess.run(
        ["bash", script_path],
        cwd=mol_path,
        capture_output=True,
        text=True
    )

    print(f"Submitted {mol}:")
    print(result.stdout)
    print(result.stderr)

## 4.3 Run QCxMS postprocessing: check that number of successful frag. runs was more than 90 %,
## and generate a raw spectra (unfiltered, with floats for m/z numbers)

In [2]:
import os
import subprocess

sim_dir = "../data/simulation_results/toy_dataset_tms"
script_path = "../../../src/workflow/submit_qcxms_fragmentation.sh"

result = subprocess.run(["bash", script_path], cwd=sim_dir,
                        capture_output=True, text=True)

print(result.stdout)
print(result.stderr)


Submitted batch job 30806746
Submitted SLURM array job (toy_dataset_tms_multi.slurm) for 4 directories




## 5.1 Run NEIMS for each molecule using the optimized SDF

## 5.2 Run CFM-ID for compounds it is valid for 

## 6. Compare NEIMS and QCxMS spectra to eachother and to the reference spectra (choices, binning all peaks to closest integer, then removing peaks with X % of base peak intensity)

## 6.1 Test only using 20 most strong signals 

## 7. Plot results